# Preprocessing of avalanche events for dataviz in Tableau

In [1]:
# importing standard Python libraries for data analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
# I would like to see all rows and columns of dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# checking our dataset
df_aval = pd.read_excel("aval_alps_2010_2019_org.xlsx")
df_aval.head()

,Id,Pays,Massif,Sommet/secteur,itinéraire,Orientation,Date,year,month,day,date_format,Heure,Description,Décl à distance,Distance décl,Caractéristique,Dénivelé,Origine principale,Origine secondaire,Alti départ,Commentaire zone départ,Epaisseur rupture,Epaisseur max rupture,Longueur rupture,Ecoulement principal,Commentaire type écoulement,Alti arrivée,Commentaire zone arrivée,Qualité neige,Commentaire qualité neige,Commentaire qualité transportée,Risque MF
0,1557382479001,France,Aiguilles de l'Argentiere/ Sept Laux,col de l'Amiante,Départ de Rieux Claret pour direction du Roche...,E,07/05/2019,2019,5,7,2019-05-07,11:00,Départ provoqué par une skieuse lors de la des...,False,0,NaN,0,Skieur rando. descente,Neige,2750,NaN,0,50,0,NaN,NaN,2700,NaN,NaN,NaN,NaN,Inconnu
1,50000560,France,Aiguilles Rouges,L'Index,NaN,SE,18/12/2010,2010,12,18,2010-12-18,13:00:00,Une plaque friable a été déclenchée par un ski...,False,0,Plaque friable,200,Skieur hors piste,NaN,2150,L'épaisseur de la rupture était de 40 à 80 cm ...,40,80,20,NaN,NaN,1950,NaN,NaN,NaN,NaN,3 - MARQUE
2,50000693,France,Aiguilles Rouges,Brévent,Combe de Charlanon,T,23/01/2012,2012,1,23,2012-01-23,NaN,Cette plaque décrite comme &quot;monstrueuse&q...,False,0,Plaque de fond,0,Inconnue,NaN,2000,NaN,0,0,0,Dense,NaN,0,NaN,NaN,NaN,NaN,2 - LIMITE
3,1390063748315,France,Aiguilles Rouges,Col de Bérard,Traversée Crochues - Bérard,W,18/01/2014,2014,1,18,2014-01-18,NaN,Un petit incident qui remet les pendules à l'h...,False,0,Plaque à vent,150,Skieur rando. montée,NaN,2400,NaN,0,0,0,NaN,NaN,0,NaN,NaN,séche dense,NaN,3 - MARQUE
4,1426167935840,France,Aiguilles Rouges,Flégère,Lac Blanc,SE,06/03/2015,2015,3,6,2015-03-06,NaN,Avalanche probablement partie spontanément. <b...,False,0,Plaque à vent,0,NaN,NaN,2280,NaN,0,0,0,Dense,NaN,0,NaN,Sèche dure,NaN,NaN,3 - MARQUE


In [4]:
# deeper look on all variables
df_aval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Id                               1167 non-null   int64         
 1   Pays                             1167 non-null   object        
 2   Massif                           1167 non-null   object        
 3   Sommet/secteur                   1122 non-null   object        
 4   itinéraire                       904 non-null    object        
 5   Orientation                      1167 non-null   object        
 6   Date                             1167 non-null   object        
 7   year                             1167 non-null   int64         
 8   month                            1167 non-null   int64         
 9   day                              1167 non-null   int64         
 10  date_format                      1167 non-null   datetime64[

In [5]:
# viewing values of variable
df_aval.Pays.value_counts()

France    1167
Name: Pays, dtype: int64

In [6]:
# viewing names of columns for easier copypaste later
df_aval.columns

Index(['Id', 'Pays', 'Massif', 'Sommet/secteur', 'itinéraire', 'Orientation',
       'Date', 'year', 'month', 'day', 'date_format', 'Heure', 'Description',
       'Décl à distance', 'Distance décl', 'Caractéristique', 'Dénivelé',
       'Origine principale', 'Origine secondaire', 'Alti départ',
       'Commentaire zone départ', 'Epaisseur rupture', 'Epaisseur max rupture',
       'Longueur rupture', 'Ecoulement principal',
       'Commentaire type écoulement', 'Alti arrivée',
       'Commentaire zone arrivée', 'Qualité neige',
       'Commentaire qualité neige', 'Commentaire qualité transportée',
       'Risque MF'],
      dtype='object')

In [7]:
# viewing distribution of values
df_aval["Caractéristique"].value_counts()

Plaque de fond        135
Plaque à vent         122
Plaque friable        108
Départ ponctuel        21
Plaque dure             8
Plaque sur glacier      7
Sur-avalanche           1
Name: Caractéristique, dtype: int64

In [8]:
# recoding variable to English
df_aval["Caractéristique"] = df_aval["Caractéristique"].apply(lambda x: 
                                     "bottom layer" if x == "Plaque de fond"
                                     else "windy slope" if x == "Plaque à vent"
                                     else "fragile layer" if x == "Plaque friable"
                                    else "one-off event" if x == "Départ ponctuel"
                                    else "hard layer" if x == "Plaque dure"
                                     else "glacial layer" if x == "Plaque sur glacier"
                                      else np.nan)

In [9]:
# viewing distribution of values
df_aval["Décl à distance"].value_counts()

False    1082
True       85
Name: Décl à distance, dtype: int64

In [10]:
# viewing distribution of values
df_aval["Longueur rupture"].value_counts()

0       863
50       35
100      34
30       32
150      25
80       22
20       20
200      17
300      16
40       15
70       13
10       10
60       10
250       7
25        7
35        6
15        5
500       4
400       3
120       3
350       2
1500      2
5         2
8         1
4         1
2         1
2000      1
45        1
75        1
125       1
160       1
180       1
450       1
700       1
800       1
1400      1
90        1
Name: Longueur rupture, dtype: int64

In [11]:
# viewing distribution of values
df_aval["Qualité neige"].value_counts()

Sèche poudreuse    204
Humide             152
Sèche dure          34
Mouillée            31
Aucun                8
Name: Qualité neige, dtype: int64

In [12]:
# recoding variable to English
df_aval["Qualité neige"] = df_aval["Qualité neige"].apply(lambda x: 
                                     "dry powder-like snow" if x == "Sèche poudreuse"
                                     else "humid snow" if x == "Humide"
                                     else "hard dry snow" if x == "Sèche dure"
                                    else "wet snow" if x == "Mouillée"
                                     else np.nan)

In [13]:
# viewing distribution of values
df_aval["Dénivelé"].value_counts()

0       1005
200       23
100       21
300       17
150       12
500        8
400        7
120        6
350        6
250        5
80         4
40         3
50         3
60         3
110        3
1200       2
130        2
20         2
800        2
450        2
70         2
600        2
700        2
30         2
550        2
25         1
90         1
10         1
170        1
1400       1
180        1
220        1
230        1
1260       1
310        1
320        1
390        1
570        1
580        1
620        1
650        1
710        1
750        1
1000       1
1100       1
270        1
Name: Dénivelé, dtype: int64

In [14]:
# viewing distribution of values
df_aval["Origine principale"].value_counts()

Réchauffement                      166
Skieur rando.  descente            159
Skieur hors piste                  133
Neige                              108
Skieur rando.  montée               98
Inconnue                            47
Surfeur                             25
GAZEX                               23
Vent                                21
Grenadage hélico                    20
Grenadage à main                    19
Pluie                               15
Alpiniste                           13
Minage de corniche                  10
Piéton                              10
Engin de damage                      6
CATEX                                6
Raquettiste                          5
Autres                               5
Chute de sérac                       5
Autre avalanche                      2
Engin de déneigement                 2
Avalancheur                          2
Speed riding                         1
Déclenchement de corniche à ski      1
Name: Origine principale,

In [15]:
# recoding variable to English
df_aval["Origine principale"] = df_aval["Origine principale"].apply(lambda x: 
                                    "ski alpinisme-downhill" if x == "Skieur rando.  descente"
                                    else "ski alpinisme-uphill" if x == "Skieur rando.  montée"
                                    else "ski outside of piste" if x == "Skieur hors piste"
                                    else "unknown" if x == "Inconnue"
                                    else "snowboard" if x == "Surfeur"
                                    else "avalanche control system (GATEX)" if x == "GAZEX"
                                    else "bombing from helicopter" if x == "Grenadage hélico"
                                    else "tossing explosives manually" if x == "Grenadage à main"
                                    else "alpiniste" if x == "Alpiniste"
                                    else "hiking" if x == "Piéton"
                                    else "mining" if x == "Minage de corniche"
                                    else "snowshoes trip" if x == "Raquettiste"
                                    else "fall of block of glacial ice" if x == "Chute de sérac"
                                    else "snowshoes trip" if x == "Raquettiste"
                                    else "CATEX blasting (cable carrying explosives)" if x == "CATEX"
                                    else "wind" if x == "Vent"
                                    else "snow grooming" if x == "Engin de damage"
                                    else "snow grooming" if x == "Engin de déneigement"
                                    else "rain" if x == "Pluie"
                                     else np.nan)

In [16]:
# viewing distribution of values
df_aval["Origine secondaire"].value_counts()

Neige                    67
Réchauffement            45
Vent                     40
Pluie                     7
Raquettiste               1
Skieur rando.  montée     1
Name: Origine secondaire, dtype: int64

In [17]:
# recoding variable to English
df_aval["Origine secondaire"] = df_aval["Origine secondaire"].apply(lambda x: 
                                     "snow" if x == "Neige"
                                     else "warming" if x == "Réchauffement"
                                     else "wind" if x == "Vent"
                                    else "rain" if x == "Pluie"
                                     else np.nan)

In [18]:
# viewing distribution of values
df_aval["Alti départ"].value_counts()

0        244
2300      47
2200      42
2400      42
2600      39
2700      39
2500      32
2000      31
2900      28
2650      26
3000      26
2450      25
2750      25
2350      21
2550      20
2800      20
2100      18
2050      16
2150      15
2250      14
1900      12
1850      11
2950      10
1950      10
3100       9
3350       9
1800       9
3200       8
2480       7
3600       7
1700       7
2420       6
2320       6
2850       6
2170       6
2280       6
3250       5
2460       5
2380       5
3300       5
2620       5
2670       5
1750       5
3020       5
3550       5
1930       4
3050       4
2370       4
2680       4
2660       4
2080       4
2520       4
3400       4
2330       4
3500       3
2635       3
2390       3
2290       3
1940       3
3030       3
2470       3
1600       3
1690       3
2360       3
2530       3
2510       3
2560       3
2580       2
2130       2
2570       2
2030       2
1650       2
2770       2
2760       2
1820       2
2175       2
2440       2

In [19]:
# viewing distribution of values
df_aval["Epaisseur rupture"].value_counts()

0      852
30      52
50      43
100     41
20      39
40      35
80      23
60      17
150     12
10      10
200      8
70       7
25       5
15       5
130      4
120      2
300      2
250      2
5        2
2        2
90       1
140      1
12       1
500      1
Name: Epaisseur rupture, dtype: int64

In [20]:
# viewing distribution of values
df_aval["Ecoulement principal"].value_counts()

Dense      426
Mixte       83
Aérosol     19
Name: Ecoulement principal, dtype: int64

In [21]:
# recoding variable to English
df_aval["Ecoulement principal"] = df_aval["Ecoulement principal"].apply(lambda x: 
                                     "dense" if x == "Dense"
                                     else "mixed" if x == "Mixte"
                                     else "aerosol" if x == "Aérosol"
                                     else np.nan)

In [22]:
# getting rid of not needed data for visualization
df_aval = df_aval.drop(columns=['Pays', 'Id', 'Commentaire qualité transportée', 
                                 'Commentaire type écoulement', 'Commentaire qualité neige',
                                'Commentaire zone arrivée', 'Commentaire zone départ'])

In [23]:
# viewing names of columns
df_aval.columns

Index(['Massif', 'Sommet/secteur', 'itinéraire', 'Orientation', 'Date', 'year',
       'month', 'day', 'date_format', 'Heure', 'Description',
       'Décl à distance', 'Distance décl', 'Caractéristique', 'Dénivelé',
       'Origine principale', 'Origine secondaire', 'Alti départ',
       'Epaisseur rupture', 'Epaisseur max rupture', 'Longueur rupture',
       'Ecoulement principal', 'Alti arrivée', 'Qualité neige', 'Risque MF'],
      dtype='object')

In [24]:
# renaming variables to English
df_aval.columns = ['massif', 'mountain/area', 'plan_of_trip', 'orientation', 'date', 'year',
       'month', 'day', 'date_format', 'hour', 'description',
       'remote_avalanche_triggering', 'distance_of_remote_trigger', 'characteristics', 'elevation_gain',
       'main_cause_of_avalanche', 'secondary_cause_of_avalanche', 'altitude_of_depart',
       'thickness_of_avalanche_breakdown', 'max_thickness_of_avalanche_breakdown', 'length_of_avalanche_breakdown',
       'main_snowflow', 'altitude_of_arrival', 'snow_quality', 'risque_mf']

In [25]:
# checking the result
df_aval.head()

,massif,mountain/area,plan_of_trip,orientation,date,year,month,day,date_format,hour,description,remote_avalanche_triggering,distance_of_remote_trigger,characteristics,elevation_gain,main_cause_of_avalanche,secondary_cause_of_avalanche,altitude_of_depart,thickness_of_avalanche_breakdown,max_thickness_of_avalanche_breakdown,length_of_avalanche_breakdown,main_snowflow,altitude_of_arrival,snow_quality,risque_mf
0,Aiguilles de l'Argentiere/ Sept Laux,col de l'Amiante,Départ de Rieux Claret pour direction du Roche...,E,07/05/2019,2019,5,7,2019-05-07,11:00,Départ provoqué par une skieuse lors de la des...,False,0,NaN,0,ski alpinisme-downhill,snow,2750,0,50,0,NaN,2700,NaN,Inconnu
1,Aiguilles Rouges,L'Index,NaN,SE,18/12/2010,2010,12,18,2010-12-18,13:00:00,Une plaque friable a été déclenchée par un ski...,False,0,fragile layer,200,ski outside of piste,NaN,2150,40,80,20,NaN,1950,NaN,3 - MARQUE
2,Aiguilles Rouges,Brévent,Combe de Charlanon,T,23/01/2012,2012,1,23,2012-01-23,NaN,Cette plaque décrite comme &quot;monstrueuse&q...,False,0,bottom layer,0,unknown,NaN,2000,0,0,0,dense,0,NaN,2 - LIMITE
3,Aiguilles Rouges,Col de Bérard,Traversée Crochues - Bérard,W,18/01/2014,2014,1,18,2014-01-18,NaN,Un petit incident qui remet les pendules à l'h...,False,0,windy slope,150,ski alpinisme-uphill,NaN,2400,0,0,0,NaN,0,NaN,3 - MARQUE
4,Aiguilles Rouges,Flégère,Lac Blanc,SE,06/03/2015,2015,3,6,2015-03-06,NaN,Avalanche probablement partie spontanément. <b...,False,0,windy slope,0,NaN,NaN,2280,0,0,0,dense,0,hard dry snow,3 - MARQUE


In [26]:
# viewing different values for hour
df_aval.hour.value_counts()

12:00                              41
11:00                              36
14:00                              32
12:30                              29
15:00                              26
13:30                              22
16:00                              20
13:00                              19
10:30                              19
11:30                              18
10:00                              16
09:00                              12
13:00:00                            9
14:30                               9
17:30                               9
15:30                               9
08:00                               8
10:30:00                            7
08:30                               7
17:00                               6
10:15                               6
15:00:00                            6
12:00:00                            6
18:00                               5
11:00:00                            5
14:45                               5
16:30       

In [27]:
# deleting leading and trailing spaces 
df_aval.hour = df_aval.hour.str.strip()

In [28]:
# unifying different writing of same time expressions
df_aval['hour'].replace(['h', 'H', '/', ';'], ':', inplace=True, regex=True)
df_aval['hour'].replace([' : ', ':  '], ':', inplace=True, regex=True)

In [29]:
# keeping only hours and minutes
df_aval.hour = df_aval["hour"].str[0:5]

In [30]:
# replacing spaces inside the values
df_aval.hour = df_aval.hour.str.replace(' ', '')

In [31]:
# filling shorter values with 0 to have unified length of value to 5 chars
df_aval.hour = df_aval["hour"].str.ljust(5, '0')

In [32]:
# checking values after all transformations to see if more changes are needed
df_aval.hour.value_counts()

12:00    49
11:00    45
14:00    37
15:00    33
12:30    31
13:00    30
13:30    26
10:30    26
11:30    25
16:00    24
10:00    20
09:00    15
15:30    14
14:30    12
17:30    12
08:00    10
08:30     9
17:00     9
11:45     8
10:15     7
18:00     7
16:30     7
14:45     6
09:30     5
14:15     5
15:45     5
11:15     5
12:15     4
14:25     4
13:45     4
14:40     4
10:45     4
11:10     4
13:40     3
19:00     3
15:50     3
06:00     3
12:45     3
16:15     3
15:20     3
11:40     3
13:15     3
08:15     3
20:00     3
16:45     2
16:10     2
12:10     2
14:10     2
13:50     2
16:35     2
10.00     2
16:40     2
04:00     2
13:20     2
09:15     2
07:40     2
10:20     2
9:300     2
15:25     2
14:20     2
12:50     2
08:45     2
8:000     2
13:10     2
17:20     1
10:25     1
21:00     1
06:20     1
14:11     1
8:300     1
10:58     1
09:50     1
14:39     1
90000     1
?0000     1
11000     1
10:05     1
09:24     1
15:15     1
12:06     1
Avant     1
13000     1
02:08     1
12:2

In [33]:
# last changes for times still not fitting the unified version
df_aval['hour'].replace('000', ':00', inplace=True, regex=True)
df_aval['hour'].replace(['9::00', '9:000'], '09:00', inplace=True, regex=True)
df_aval['hour'].replace(['8::00', '8:000'], '08:00', inplace=True, regex=True)
df_aval['hour'].replace('14:à0', '14:00', inplace=True, regex=True)
df_aval['hour'].replace('9:300', '09:00', inplace=True, regex=True)
df_aval['hour'].replace('6::00', '06:00', inplace=True, regex=True)
df_aval['hour'].replace('8:300', '08:30', inplace=True, regex=True)
df_aval['hour'].replace('.00', ':00', inplace=True, regex=True)
df_aval['hour'].replace(['Début', 'Avant', 'Incon'], np.nan, inplace=True, regex=True)
df_aval['hour'].replace('\?:000', np.nan, inplace=True, regex=True)

In [34]:
# verifying all hours are written in same manner
df_aval.hour.value_counts()

12:00    49
11:00    47
14:00    38
15:00    33
13:00    31
12:30    31
10:30    26
13:30    26
11:30    25
16:00    24
10:00    22
09:00    19
15:30    14
17:30    12
08:00    12
14:30    12
08:30    10
17:00     9
11:45     8
16:30     7
18:00     7
10:15     7
14:45     6
15:45     5
14:15     5
09:30     5
11:15     5
10:45     4
14:40     4
06:00     4
11:10     4
12:15     4
13:45     4
14:25     4
11:40     3
13:15     3
08:15     3
12:45     3
20:00     3
16:15     3
15:50     3
19:00     3
13:40     3
15:20     3
09:15     2
13:50     2
16:40     2
12:50     2
14:10     2
13:10     2
08:45     2
10:20     2
12:10     2
04:00     2
14:20     2
16:10     2
13:20     2
16:45     2
15:25     2
07:40     2
16:35     2
09:24     1
05:15     1
02:08     1
12:20     1
10:25     1
06:20     1
10:34     1
15:15     1
14:39     1
16:20     1
01:00     1
05:00     1
14:05     1
12:06     1
17:20     1
07:30     1
11:25     1
09:50     1
09:25     1
11:55     1
15:10     1
14:11     1
18:3

In [35]:
# creating new variable only for capturing hour
df_aval["only_hour"] = df_aval.hour.str[0:2]

In [36]:
# viewing distribution of values
df_aval.risque_mf.value_counts()

3 - MARQUE       464
Inconnu          352
2 - LIMITE       190
4 - FORT         113
1 - FAIBLE        33
5 - TRES FORT     15
Name: risque_mf, dtype: int64

In [37]:
# changing data type of the variable from string to int
df_aval.risque_mf = df_aval.risque_mf.str[0:1]
df_aval.risque_mf.replace('I', np.nan, inplace=True, regex=True)
df_aval.risque_mf = (df_aval.risque_mf.fillna(0)).astype(int)

In [38]:
# changing data type of the variable from string to int
df_aval.only_hour = (df_aval.only_hour.fillna(0)).astype(int)

In [39]:
# last check of all variables and its data types
df_aval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   massif                                1167 non-null   object        
 1   mountain/area                         1122 non-null   object        
 2   plan_of_trip                          904 non-null    object        
 3   orientation                           1167 non-null   object        
 4   date                                  1167 non-null   object        
 5   year                                  1167 non-null   int64         
 6   month                                 1167 non-null   int64         
 7   day                                   1167 non-null   int64         
 8   date_format                           1167 non-null   datetime64[ns]
 9   hour                                  648 non-null    object        
 10  

In [40]:
# saving cleaned dataset
avalanches_for_t = df_aval.to_csv(r'.\\avalanches_for_t.csv', index = False)

In [45]:
df_aval.massif.value_counts()

Haute Maurienne                         210
Vanoise                                 148
Haute Tarentaise                        115
Maurienne                                81
Mont Blanc                               65
Aravis                                   57
Queyras                                  50
Belledonne                               45
Oisans                                   45
Beaufortain                              43
Lauzière                                 42
Ecrins                                   33
Mont Thabor                              31
Cerces                                   21
Grandes Rousses                          18
Dévoluy                                  15
Chartreuse                               15
Briançonnais                             14
Taillefer                                14
Mercantour                               13
Ubaye - Parpaillon                       12
Aiguilles Rouges                         11
Chablais                        